In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
import patsy

# Load data
cd = pd.read_csv("../Data/data.csv")

# Data preparation
cd['pair_id'] = cd['session'].astype(str) + "_" + cd['group'].astype(str)
cd['arr'] = pd.factorize(cd['arrival'])[0] + 1

# Keep only the needed columns
columns_to_keep = ["exp", "pair_id", "code", "round", "arr", "choice", "certainty"]
Data = cd[columns_to_keep]

# Convert categorical data into factors
Data['choice'] = Data['choice'].map({0: "office", 1: "canteen"})
Data['exp'] = Data['exp'].astype('category')

# Create a binary response variable for GLMM
Data['choice_bin'] = Data['choice'].apply(lambda x: 1 if x == "canteen" else 0)

# Debugging: Ensure data integrity
print("Columns in Data:", Data.columns)
print("Sample rows:\n", Data.head())

# Fixed-effects design matrix using patsy
y, X_fixed = patsy.dmatrices("choice_bin ~ arr * exp", data=Data, return_type="dataframe")

# Random-effects design matrix: One-hot encoding for `pair_id`
X_random = pd.get_dummies(Data['pair_id'], drop_first=False)

# Ensure `X_random` contains proper group-specific columns
if X_random.empty:
    raise ValueError("Random-effects design matrix `X_random` is empty.")

# Debugging outputs
print("Fixed-effects design matrix:\n", X_fixed.head())
print("Random-effects design matrix:\n", X_random.head())

# Define `ident`: Assigning the same group to all random effects
ident = [1] * X_random.shape[1]

# Fit the Binomial GLMM
model = BinomialBayesMixedGLM(Data['choice_bin'], X_fixed, X_random, ident)
result = model.fit_vb()

# Summary of results
print(result.summary())

# Calculate predicted probabilities manually
# Fixed-effects component
fixed_effects = np.dot(X_fixed, result.params[:X_fixed.shape[1]])

# Random-effects component
random_effects = np.dot(X_random, result.params[X_fixed.shape[1]:])

# Predicted log-odds
log_odds = fixed_effects + random_effects

# Convert log-odds to probabilities
Data['pred'] = 1 / (1 + np.exp(-log_odds))

# Plotting the fitted probabilities
sns.lineplot(data=Data, x="arr", y="pred", hue="exp", palette="Set1")
plt.title("Fitted Probabilities of Choosing Canteen")
plt.xlabel("Arrival Time")
plt.ylabel("Probability")
plt.show()

C:\Users\hjl161\AppData\Local\Temp\ipykernel_22852\1804807015.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data['choice'] = Data['choice'].map({0: "office", 1: "canteen"})
C:\Users\hjl161\AppData\Local\Temp\ipykernel_22852\1804807015.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data['exp'] = Data['exp'].astype('category')
C:\Users\hjl161\AppData\Local\Temp\ipykernel_22852\1804807015.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

Columns in Data: Index(['exp', 'pair_id', 'code', 'round', 'arr', 'choice', 'certainty',
       'choice_bin'],
      dtype='object')
Sample rows:
    exp     pair_id      code  round  arr   choice  certainty  choice_bin
0  AMT  0c0pdasz_2  zh74wr23      1    1  canteen      0.875           1
1  AMT  0c0pdasz_2  w73mxa5y      1    2  canteen      0.875           1
2  AMT  0c0pdasz_2  zh74wr23      2    1   office      0.500           0
3  AMT  0c0pdasz_2  w73mxa5y      2    3  canteen      0.875           1
4  AMT  0c0pdasz_2  zh74wr23      3    1  canteen      0.990           1
Fixed-effects design matrix:
    Intercept  exp[T.DTU1]  exp[T.DTU2]  arr  arr:exp[T.DTU1]  arr:exp[T.DTU2]
0        1.0          0.0          0.0  1.0              0.0              0.0
1        1.0          0.0          0.0  2.0              0.0              0.0
2        1.0          0.0          0.0  1.0              0.0              0.0
3        1.0          0.0          0.0  3.0              0.0             

ValueError: shapes (7432,401) and (403,) not aligned: 401 (dim 1) != 403 (dim 0)